<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Defcon4_TFLite_Mobilenet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
try: shutil.rmtree(('/content/sample_data'))
except: pass
print('Updated: 11/15/2022-3')


In [ ]:
try:
    import tflite_model_maker
    print('tflite_model_maker:', tflite_model_maker.__version__)
except BaseException as err:
    print(err, '\n')
    !sudo apt -y install libportaudio2
    print()
    !sudo apt autoremove libnvidia-common-460
    print()
    !pip install coremltools
    print()
    !pip install tflite-model-maker

In [ ]:
# https://www.tensorflow.org/tfmodels/vision/image_classification
import tensorflow as tf

logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices()]

# if exp_config.runtime.mixed_precision_dtype == tf.float16: tf.keras.mixed_precision.set_global_policy('mixed_float16')

if 'GPU' in ''.join(logical_device_names):
    distributionStrategy = tf.distribute.MirroredStrategy()
    with distributionStrategy.scope():
        print('Running on GPU')

elif 'TPU' in ''.join(logical_device_names):
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print(tpu)
    tf.config.experimental_connect_to_cluster(tpu)
    tf.distribute.TPUStrategy(tpu)
    distributionStrategy = tf.distribute.TPUStrategy(tpu)
    with distributionStrategy.scope():
        print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

else:
    distributionStrategy = tf.distribute.OneDeviceStrategy(logical_device_names[0])
    with distributionStrategy.scope():
        print('Running on CPU')
        print(f'Warning: this will be very slow.')

In [ ]:
from __future__ import absolute_import
import os, shutil, tarfile 
from os.path import abspath, exists, join
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

from google.colab import drive, files
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

contentPath=os.getcwd()

if exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

drive.mount('/content/drive', force_remount=True)

# import required modules
if not exists('BashColors.py'):
    shutil.copy2(src='/content/drive/MyDrive/PythonFiles/BashColors.py',
                 dst=contentPath)
    
if not exists('TarfileFunctions.py'):
    shutil.copy2(src='/content/drive/MyDrive/TarfileFunctions.py',
                 dst=contentPath)

# if not exists('GetTPU.py'):
    # shutil.copy2(src='/content/drive/MyDrive/PythonFiles/GetTPU.py',
                 # dst=contentPath)
    
if not exists('ExceptionHandeler.py'):
    shutil.copy2(src='/content/drive/MyDrive/PythonFiles/ExceptionHandeler.py',
                 dst=contentPath)
    
from BashColors import C
# from GetTPU import *
from TarfileFunctions import *
from ExceptionHandeler import *

# create image directories 
if not exists('DataGenerator7.tar.gz'):
    shutil.copy2(src='/content/drive/MyDrive/DataGenerator7.tar.gz',
                 dst=contentPath)
    os.chdir(contentPath)
    tff.extractTarfiles('DataGenerator7.tar.gz')

if not exists('images.tar.gz'):
    shutil.copy2(src='/content/drive/MyDrive/images.tar.gz',
                 dst=contentPath)
    os.chdir(contentPath)
    tff.extractTarfiles('/content/drive/MyDrive/images.tar.gz')
    
image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')

In [ ]:
q

In [ ]:
testPath='/content/images'
print(testPath)
test_data=DataLoader.from_folder(filename=testPath, shuffle=True)

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in blue color.
try:
    plt.figure(figsize=(20, 20))
    predicts = model.predict_top_k(test_data)
    for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
        ax = plt.subplot(10, 10, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image.numpy(), cmap=plt.cm.gray)
        
        predict_label = predicts[i][0][0]
        color = get_label_color(predict_label,
                                test_data.index_to_label[label.numpy()])
        ax.xaxis.label.set_color(color)
        plt.xlabel('Predicted: %s' % predict_label)
    plt.show()
except: pass

from tensorflow_examples.lite.model_maker.core.data_util.image_dataloader import ImageClassifierDataLoader
def plotDatasetImages(dataset:ImageClassifierDataLoader):
    """ """
    plt.figure(figsize=(12,12))
    for i, (image, label) in enumerate(dataset.gen_dataset().unbatch().take(25)):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image.numpy(), cmap=plt.cm.gray)
        plt.xlabel(data.index_to_label[label.numpy()])
    plt.show()

# plotDatasetImages(dataset=train_ds)

In [ ]:
fileGlob= glob.glob('**', recursive=True)
for fil in fileGlob:
    if fil.__contains__('.ipynb_checkpoints'):
        print(fil)

In [ ]:

# tff.tarfileFromDirectory(output_filename='DataGenerator7.tar.gz',source_dir='/content/DataGenerator')
# shutil.copy2('/content/DataGenerator7.tar.gz', '/content/drive/MyDrive')

In [ ]:
BATCH_SIZE = 16
data = DataLoader.from_folder(filename=generatorPath, shuffle=True)
train_ds, restData = data.split(0.8)
val_ds, test_ds = restData.split(0.5)
# if train_ds.index_to_label.__contains__('.ipynb_checkpoints'):
    # train_ds.index_to_label.remove('.ipynb_checkpoints')

print()
print('classes:', train_ds.num_classes)
print('class names', train_ds.index_to_label)
print(len(train_ds) // BATCH_SIZE, len(val_ds) // BATCH_SIZE)
print(train_ds.size // BATCH_SIZE, val_ds.size // BATCH_SIZE)

In [ ]:
help(image_classifier.ImageClassifier.predict_top_k)


In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

In [ ]:
from tflite_model_maker import image_classifier
BATCH_SIZE = 16
EPOCHS = 12
DROPOUT_RATE = 0.1
LEARNING_RATE = 0.0001
WARMUP = len(train_ds) // BATCH_SIZE
print('WARMUP:', WARMUP)

model = image_classifier.create(
    train_data = train_ds,
    validation_data = val_ds,
    model_spec = model_spec.get('mobilenet_v2'),
    learning_rate = LEARNING_RATE,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    dropout_rate = DROPOUT_RATE,
    use_augmentation = True, # default=False
    use_hub_library = True,  # default=True
    momentum = 0.9, # Only used when use_hub_library is True
    shuffle = True, # default=False
    warmup_steps = WARMUP,
    model_dir = contentPath,
    train_whole_model = False,
)

In [ ]:
loss, accuracy = model.evaluate(data=test_ds)
print(loss)
accuracyPercent=round(accuracy * 100, 1)
print(accuracyPercent, '% accuracy')

In [ ]:
model.export(export_dir='.')

In [ ]:
model.ALLOWED_EXPORT_FORMAT

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(12, 20))
predicts = model.predict_top_k(test_ds)
for i, (image, label) in enumerate(test_ds.gen_dataset().unbatch().take(50)):
    ax = plt.subplot(10, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image.numpy(), cmap=plt.cm.gray)

    predict_label = predicts[i][0][0]
    percent_label = predicts[i][0][1] * 100
    percent_label = round(percent_label, 1)
    color = get_label_color(predict_label,
                            val_ds.index_to_label[label.numpy()])
    ax.xaxis.label.set_color(color)
    # plt.xlabel('Predicted: %s' % predict_label)
    plt.xlabel('%s %s pct'  % (predict_label, percent_label))
plt.show()
print()

In [ ]:
type(predict_label)
predicts[i][0][1]


In [ ]:
model.export(export_dir='.',
             tflite_filename='Defcon_MobileNetV2.tflite')

In [ ]:
config = QuantizationConfig.for_float16()

model.export(export_dir='.',
             tflite_filename='Defcon_MobileNetV2_fp16.tflite',
             quantization_config=config)

In [ ]:
testData = DataLoader.from_folder(image_path, shuffle=True)
print('\n', testData, '\n')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab.patches import cv2_imshow
print(cv2_imshow)
from os.path import abspath, basename
from time import sleep
os.chdir('/content/DataGenerator')
dataGlob=glob.glob('**', recursive=True)
count=0
for pth in dataGlob:
        fullPath=abspath(pth)
        if os.path.isfile(fullPath):
            count+=1
            print(count, basename(fullPath))
            img=cv2.imread(fullPath, cv2.IMREAD_COLOR)
            # cv2_imshow(img)
            # sleep(0.2)
            img=None
            print()
os.chdir(contentPath)

In [ ]:
shutil.copy2('/content/model.tflite',
             '/content/drive/MyDrive/TensorflowModels')

shutil.copy2('/content/Defcon_MobileNetV2.tflite',
             '/content/drive/MyDrive/TensorflowModels')

shutil.copy2('/content/Defcon_MobileNetV2_fp16.tflite',
             '/content/drive/MyDrive/TensorflowModels')


In [ ]:
# !pip install -q coremltools
from ExceptionHandeler import *
try:
    import coremltools as ct
    tfmodel = tf.keras.applications.MobileNet()
    mlmodel = ct.convert('auto', "/content/model.tflite")
except: exh.printException()

In [ ]:
help(ct.convert)

In [ ]:
dir(model)

In [ ]:
import coremltools as ct 
model = MobileNet_flowers (1).h5
# float 16 input and output of type multiarray
try:
    mlmodel = ct.convert(
        model,
        inputs=[ct.TensorType(shape=input.shape, dtype=np.float16)],
        outputs=[ct.TensorType(dtype=np.float16)],
        minimum_deployment_target=ct.target.iOS16,)
except:
    exh.printException()

In [ ]:
# Tested with TensorFlow 2.6.2
import tensorflow as tf
import coremltools as ct
bs="""
tf_keras_model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ]
)
"""
tf_keras_model = model

# Pass in `tf.keras.Model` to the Unified Conversion API
mlmodel = ct.convert(tf_keras_model, convert_to="mlprogram")

# or save the keras model in SavedModel directory format and then convert
tf_keras_model.save('tf_keras_model')
mlmodel = ct.convert('tf_keras_model', convert_to="mlprogram")

# or load the model from a SavedModel and then convert
tf_keras_model = tf.keras.models.load_model('tf_keras_model')
mlmodel = ct.convert(tf_keras_model, convert_to="mlprogram")

# or save the keras model in HDF5 format and then convert
tf_keras_model.save('tf_keras_model.h5')
mlmodel = ct.convert('tf_keras_model.h5', convert_to="mlprogram")

In [ ]:
shutil.copy2('/content/drive/MyDrive/MobileNet_flowers.h5',
             contentPath)


new_model = tf.keras.models.load_model('/content/MobileNet_flowers.h5')


# Check its architecture
# new_model.summary()

import coremltools as ct 
mlmodel=None
# float 16 input and output of type multiarray
try:
    mlmodel = ct.convert(
        new_model,
        inputs=[ct.TensorType(dtype=np.float16)],
        outputs=[ct.TensorType(dtype=np.float16)],
        minimum_deployment_target=ct.target.iOS16,)
except:
    exh.printException()

In [ ]:
print(mlmodel)